# Test additive seperability of MLP model

## Import packages

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from datetime import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Is there a GPU available: 
[]


## Create dataset

We use $f(x_0,x_1) =x_0^2+x_1^3$ as our testing function. It has an additive separability property.

There are 6 sets of input data with different distribution.

In [6]:
def f(x):
    return x[0] **2 + x[1] ** 3

x_all = np.mgrid[-5:5:0.25, -5:5.25:0.25].reshape(2,-1).T
x_all_1 = np.random.normal(0, 2, size=(1640, 2))
x_all_2 = np.random.normal(0, 1.5, size=(1640, 2))
x_all_3 = np.random.normal(0, 1, size=(1640, 2))
x_all_4 = np.random.normal(0, 0.5, size=(1640, 2))
x_all_5 = np.random.normal(0, 0.05, size=(1640, 2))

y_all=np.array([f(x) for x in x_all])
y_all_1=np.array([f(x) for x in x_all_1])
y_all_2=np.array([f(x) for x in x_all_2])
y_all_3=np.array([f(x) for x in x_all_3])
y_all_4=np.array([f(x) for x in x_all_4])
y_all_5=np.array([f(x) for x in x_all_])

## Method 1.  Auto differentiation on MLP model

### Step 1. Create MLP and training

The MLP model used in the experiments is set with 1 hidden layer and 50 nodes. For calculation convenience, we choose softplus function ($\ln(1+e^{x})$) as the activation function. The detail of the model is shown in the code below. 

In [8]:
from random import random
import timeit
from statistics import mean,stdev

model = keras.Sequential([
layers.Dense(50, activation=tf.keras.activations.softplus, input_shape=[2,]),
layers.Dense(1)])

optimizer = tf.keras.optimizers.RMSprop(0.01)

loss_fn = tf.keras.losses.MeanSquaredError()

early_stopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0
)

model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

history = model.fit(
  x_all, y_all,
  epochs=5000, validation_split = 0.2, verbose=1,
             callbacks=[early_stopping])



Epoch 1/5000
41/41 [==============================] - 0s 3ms/step - loss: 1701.6582 - mae: 26.8414 - mse: 1701.6582 - val_loss: 1543.0806 - val_mae: 30.4718 - val_mse: 1543.0806
Epoch 2/5000
41/41 [==============================] - 0s 1ms/step - loss: 595.5736 - mae: 18.6498 - mse: 595.5736 - val_loss: 905.1691 - val_mae: 24.2762 - val_mse: 905.1691
Epoch 3/5000
41/41 [==============================] - 0s 1ms/step - loss: 491.0971 - mae: 18.6858 - mse: 491.0971 - val_loss: 799.8300 - val_mae: 22.7144 - val_mse: 799.8300
Epoch 4/5000
41/41 [==============================] - 0s 1ms/step - loss: 470.4480 - mae: 18.3095 - mse: 470.4480 - val_loss: 793.4431 - val_mae: 22.6321 - val_mse: 793.4431
Epoch 5/5000
41/41 [==============================] - 0s 1ms/step - loss: 452.0608 - mae: 18.0016 - mse: 452.0608 - val_loss: 718.2554 - val_mae: 21.6236 - val_mse: 718.2554
Epoch 6/5000
41/41 [==============================] - 0s 1ms/step - loss: 435.1060 - mae: 17.8103 - mse: 435.1060 - val_loss: 

### Step 2. Calculate $\partial f / \partial \mathbf{x}$ from original MLP

In [10]:
x_tensor = tf.convert_to_tensor(x_all)
with tf.GradientTape() as tape:
    tape.watch(x_tensor)
    fx = model(x_tensor)  # compute f(x)
gradients = tape.gradient(fx, x_tensor).numpy()
print(gradients)

[[-9.18889236 56.8438797 ]
 [-9.17642593 55.27342606]
 [-9.13530445 53.22121429]
 ...
 [ 5.00059557 47.70395279]
 [ 4.99287796 48.41611099]
 [ 4.97862244 48.91444016]]


### Step 3.  Train MLP model for $\partial f / \partial x_0$ 

In [12]:
from random import random

model_d1 = keras.Sequential([
layers.Dense(50, activation=tf.keras.activations.relu, input_shape=[2,]),
layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.01)

loss_fn = tf.keras.losses.MeanSquaredError()
early_stopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0
) 

model_d1.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])
model_d1.summary()

history = model_d1.fit(
  x_all, gradients[:,0],
  epochs=5000, validation_split = 0.2, verbose=1,
             callbacks=[early_stopping])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                150       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000
41/41 [==============================] - 0s 3ms/step - loss: 2.7900 - mae: 1.1291 - mse: 2.7900 - val_loss: 5.3768 - val_mae: 2.0113 - val_mse: 5.3768
Epoch 2/5000
41/41 [==============================] - 0s 1ms/step - loss: 0.5172 - mae: 0.5786 - mse: 0.5172 - val_loss: 2.7982 - val_mae: 1.4763 - val_mse: 2.7982
Epoch 3/5000
41/41 [==============================] - 0s 1ms/step - loss: 0.3815 - mae: 0.4926 - mse: 0.3815 - val_loss: 1.5836 - val_mae: 1.0343 - val_mse: 1.5836
Epoch 4/5000
41/41 [======

### Step 4. Calculate $\partial^2 f / \partial^2 x_0 x_1$ from $\partial f / \partial x_0$ MLP 

In [13]:
from random import random

x_tensor = tf.convert_to_tensor(x_all)
with tf.GradientTape() as tape_d1:
    tape_d1.watch(x_tensor)
    fx_prime = model_d1(x_tensor)  # compute f(x
    
gradients_d1 = tape_d1.gradient(fx_prime, x_tensor).numpy()
print(gradients_d1)

[[0.7015242  0.20485801]
 [0.51277667 0.31509325]
 [0.51277667 0.31509325]
 ...
 [0.48645252 0.06375176]
 [0.48645252 0.06375176]
 [0.54313195 0.02645294]]


## Variation of method 1: differentiation by hand

After we train our first MLP model of $f(x_0,x_1)$, instead of using tensorflow auto-differentiation to take the derivative, we can compute the differentiation of the MLP model by hands.

In [41]:
W1=np.array(model.get_weights()[0])
W2=np.array(model.get_weights()[2])
bias1=np.array(model.get_weights()[1])
bias2=np.array(model.get_weights()[3])
# use Softplus so that sigma_hat is sigmoid
def softplus(x):
    return np.log(1 + np.exp(x))
def sig(x):
    return 1/(1 + np.exp(-x))
def sig_div1(x):
    return sig(x)*(1-sig(x))

bias1_new=np.repeat(bias1, 1640).reshape((50, 1640))
bias2_new=np.repeat(bias2, 1640).reshape((1, 1640))
W1_col1=W1[0,:]
W1_col2=W1[1,:]

y_hand=np.matmul(W2.T,softplus(np.matmul(W1.T,x_all.T)+bias1_new))+bias2_new
dx0_by_hand=np.matmul((W2*sig(np.matmul(W1.T,x_all.T))).T,W1_col1)
dx1_by_hand=np.matmul((W2*sig(np.matmul(W1.T,x_all.T))).T,W1_col2)

dx0x1_by_hand=np.matmul(sig_div1(np.matmul(x_all,W1)),(W2*(W1_col2*W1_col1).reshape((50,1))))
dx1x0_by_hand=np.matmul(sig_div1(np.matmul(x_all,W1)),(W2*(W1_col2*W1_col1).reshape((50,1))))

print("dx0x1_by_hand mean: ",np.mean(abs(dx0x1_by_hand)))


dx0x1_by_hand mean:  0.37636046256784417


### Method 2.  Distribute constant

Step 1. Train MLP model on the function $ f(x_0,x_1)$ (same as step 1 in method 1).

Step 2. Insert data $x_0, x_1$ and constant $c_0,c_1$ into the function $ f(x_0,x_1)+f(c_0,c_1)-f(c_0,x_1)-f(x_0,c_1)$. The expected value is 0.

In [43]:
x0 = [x[0] for x in x_all]
x1 = [x[1] for x in x_all]
c0=np.random.rand(1640)*5
c1=np.random.rand(1640)*5

model1=model(x_all,y_all)
x0_x1=model.predict(np.column_stack((x0,x1)))
x0_c1=model.predict(np.column_stack((x0,c1)))
c0_x1=model.predict(np.column_stack((c0,x1)))
c0_c1=model.predict(np.column_stack((c0,c1)))
result=x0_x1-x0_c1-c0_x1+c0_c1

absValues = [abs(number) for number in result]
print("result mean:",np.mean(absValues))

result mean: 1.4719182
